In [48]:
# ----- READING IN THE PROVIDED DATA

import csv
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import random
from   sklearn import linear_model
from   sklearn.metrics import mean_squared_error, r2_score

N_TData = 900;   # data set contains 900 elements

# Train data
ID  = np.zeros(N_TData)
y   = np.zeros(N_TData)
x1  = np.zeros(N_TData)
x2  = np.zeros(N_TData)
x3  = np.zeros(N_TData)
x4  = np.zeros(N_TData)
x5  = np.zeros(N_TData)

# Reading the training data
with open('train.csv') as csvfile:
    reader = csv.DictReader(csvfile, delimiter = ',')
    for row in reader:
        ID[int(row['Id'])] = row['Id']
        y[int(row['Id'])]  = row['y']
        x1[int(row['Id'])] = row['x1']
        x2[int(row['Id'])] = row['x2']
        x3[int(row['Id'])] = row['x3']
        x4[int(row['Id'])] = row['x4']
        x5[int(row['Id'])] = row['x5']

# Transform the data accordingly

# Linear
phy_1  = x1
phy_2  = x2
phy_3  = x3
phy_4  = x4
phy_5  = x5

# Quadratic
phy_6  = np.square(x1)
phy_7  = np.square(x2)
phy_8  = np.square(x3)
phy_9  = np.square(x4)
phy_10 = np.square(x5)

# Exponential
phy_11 = np.exp(x1)
phy_12 = np.exp(x2)
phy_13 = np.exp(x3)
phy_14 = np.exp(x4)
phy_15 = np.exp(x5)

# Cosine
phy_16 = np.cos(x1)
phy_17 = np.cos(x2)
phy_18 = np.cos(x3)
phy_19 = np.cos(x4)
phy_20 = np.cos(x5)

# Constant
phy_21 = np.ones(phy_20.shape)

X_data = (np.vstack((phy_1, phy_2, phy_3, phy_4, phy_5, phy_6, phy_7, phy_8, phy_9, phy_10, phy_11, phy_12, phy_13, phy_14, phy_15, phy_16, phy_17, phy_18, phy_19, phy_20, phy_21))).T

In [157]:
# ----- RANDOMLY SPLIT UP THE DATA INTO TRAINING AND VALIDATION DATA

# choose random indices from 0 to 899
T_index = random.sample(range(0,N_TData-1), 700)
V_index  = list(map(int,[x for x in np.linspace(0,899,900) if x not in T_index]));

# extract the corresponding data
X_train = X_data[T_index,:]
X_vali  = X_data[V_index,:]

y_train = y[T_index]
y_vali  = y[V_index]

[len(T_index), len(V_index)]

[700, 200]

In [158]:
# ----- TRAINING

# Create linear regression object
regr = linear_model.LinearRegression()
# Train the model using the training set
regr.fit(X_train,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [159]:
# ----- VALIDATION

Y_pred = regr.predict(X_vali)

RMSE     = mean_squared_error(y_vali, Y_pred)**0.5
print(RMSE)

10.59776973617782
